In [1]:
from clase_sms_ccd import Clase_SMS
from datetime import datetime, timedelta
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

excel_rutas = "RUTAS.xlsx"
df_rutas = pd.read_excel(excel_rutas)
fecha_hoy = datetime.today()
fecha_ayer = fecha_hoy - timedelta(days=1)
fecha_ayer = fecha_ayer.strftime("%Y%m%d")
fecha_hoy = datetime.today().strftime("%Y%m%d")
fecha_hoy_txt = datetime.today().strftime("%d.%m.%Y")
ruta_zfir60 = df_rutas["RUTA"][0]
ruta_modelo = df_rutas["RUTA"][1]
ruta_dacxanalista = df_rutas["RUTA"][2]

reporte = Clase_SMS(fecha_hoy, fecha_ayer, fecha_hoy_txt, ruta_zfir60, ruta_modelo, ruta_dacxanalista)

In [2]:
reporte.traer_archivos()
reporte.preparar_zfir60()
reporte.exportar_deudores()

In [3]:
reporte.preparar_fbl5n()
reporte.preparar_recaudacion()

In [4]:
reporte.exportar_archivos_txt()

In [ ]:
#---------------------------------------- FIN DEL PROGRAMA ----------------------------------------#

In [ ]:
#---------------------------------------- EXPORTAR TXT ----------------------------------------#

In [5]:
from datetime import datetime, timedelta
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
df_dacxanalista = pd.read_excel(ruta_dacxanalista+"/Nuevo_DACxANALISTA.xlsx", sheet_name="Base_NUEVA")
df_dacxanalista = df_dacxanalista[["DEUDOR", "TIPO_DAC", "ESTADO"]]
lista_tipo_dac_no_validos = ["PVA", "TARJETERO", "RED", "RECARGA FFVV", "PROVEEDOR", "CROSSBORDER", "CDR", "CACE", "AGENTE TELMEX", "AGENTE TELMEX  / RED", "DAC RURAL"]
df_dacxanalista = df_dacxanalista[df_dacxanalista["ESTADO"] == "OPERATIVO CON MOVIMIENTO"]
df_dacxanalista = df_dacxanalista[~df_dacxanalista["TIPO_DAC"].isin(lista_tipo_dac_no_validos)]

# Valida que los deudores de la base se encuentren operativos en el dacxanalista
deudores_dacxanalista = df_dacxanalista["DEUDOR"].tolist()
ruta_base_celulares = "./bases/Base_Celulares_CCD.xlsx"
df_base_celulares = pd.read_excel(ruta_base_celulares)
deudores_base = df_base_celulares["DEUDOR"].tolist()

no_encontrados = []
for deudor in deudores_base:
    if deudor not in deudores_dacxanalista:
        no_encontrados.append(deudor)
print("Deudores no encontrados:",len(no_encontrados))
print("Deudores encontrados:",len(deudores_base)-len(no_encontrados))

# Valida que los deudores tengan un celular valido
df_base_celulares = df_base_celulares[~df_base_celulares["DEUDOR"].isin(no_encontrados)]
df_base_celulares = df_base_celulares[["DEUDOR", "NOMBRE", "CELULAR"]]
df_base_celulares = df_base_celulares[df_base_celulares["CELULAR"]!=0]
print("Deudores con celular:", df_base_celulares.shape[0])

Deudores no encontrados: 51
Deudores encontrados: 655
Deudores con celular: 444


In [7]:
df_base_celulares.head()

,DEUDOR,NOMBRE,CELULAR
11,4070566,AQUINO DURAN EDITH,986786803
17,4070826,UMI CELL PHONE E.I.R.L.,964102128
18,4071033,WORK NET INGENIERIA Y TELECOMUNICACIONES S.A.,949073141
23,4071209,M & R MARTINEZ E.I.R.L.,949093753
30,4071923,DASSA MOVIL S.A.C.,947703738


In [8]:
df_base_celulares.shape

(444, 3)

In [ ]:
df_dacxanalista.to_excel("DACxANALISTA.xlsx",index=False)